<a href="https://colab.research.google.com/github/sarwaridas/AdversarialPatches/blob/main/src/TransferabilityExperiments.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/gdrive')

from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials


Mounted at /gdrive


In [ ]:
import torch
import torchvision
from torch import nn
from torch.functional import F
from torch.optim import *
import torchvision.transforms as transforms
from torchvision.models import resnet18
from torchvision.datasets import CIFAR10
import matplotlib.pyplot as plt
import numpy as np
import warnings
warnings.filterwarnings("ignore")

device = 'cuda' if torch.cuda.is_available() else 'cpu'
torch.cuda.is_available()

True

Downloading VGG model

In [ ]:
import torchvision.transforms as transforms
from torchvision import models
model = torchvision.models.vgg16(pretrained = True)
input_lastLayer = model.classifier[6].in_features
model.classifier[6] = nn.Linear(input_lastLayer,10)
model = model.to(device)
model.load_state_dict(torch.load('/gdrive/MyDrive/Final Project/VGG16.ckpt'))
model.eval()
print('model loaded and ready to run')

Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /root/.cache/torch/hub/checkpoints/vgg16-397923af.pth


  0%|          | 0.00/528M [00:00<?, ?B/s]

model loaded and ready to run


Preprocessing data and creating Data Loaders

In [ ]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.49139968, 0.48215827 ,0.44653124), (0.24703233, 0.24348505, 0.26158768))
    ])


batch_size = 512

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transforms.ToTensor())
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                         shuffle=False, num_workers=2)


  0%|          | 0/170498071 [00:00<?, ?it/s]

Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


In [ ]:
def imshow(img):
    #img = (img * TENSOR_STD) + TENSOR_MEANS     # unnormalize
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.show()

Writing a function for placing patch on test data

In [ ]:
NORM_MEAN = np.array([0.49139968, 0.48215827 ,0.44653124])
NORM_STD = np.array([0.24703233, 0.24348505, 0.26158768])
TENSOR_MEANS, TENSOR_STD = torch.FloatTensor(NORM_MEAN)[:,None,None], torch.FloatTensor(NORM_STD)[:,None,None]
def patch_forward(patch):
    # Map patch values from [-infty,infty] to ImageNet min and max
    patch = (torch.tanh(patch) + 1 - 2 * TENSOR_MEANS) / (2 * TENSOR_STD)
    return patch


def place_patch(img, patch):
    for i in range(img.shape[0]):
        h_offset = np.random.randint(0,img.shape[2]-patch.shape[1]-1)
        w_offset = np.random.randint(0,img.shape[3]-patch.shape[2]-1)
        img[i,:,h_offset:h_offset+patch.shape[1],w_offset:w_offset+patch.shape[2]] = patch_forward(patch)
    return img

Downloading patch from drive

In [ ]:
patch = torch.load('/gdrive/MyDrive/Final Project/truck16x16.pt')

Targeted attacks: choosing a target label and checking accuracy

In [ ]:
#targeted attacks
total_examples = 0
correct_examples = 0
target_class_count = 0
val_loss = 0 # again, track the validation loss if you want

classes_dict =   {0: 'plane', 
                  1: 'car', 
                  2: 'bird', 
                  3: 'cat', 
                  4: 'deer', 
                  5: 'dog', 
                  6: 'frog', 
                  7: 'horse', 
                  8: 'ship', 
                  9: 'truck'}

target_class=9
# disable gradient during validation, which can save GPU memory
with torch.no_grad():
  for batch_idx, (inputs, labels) in enumerate(testloader):
    # inputs, labels = data
    inputs = place_patch(inputs, patch)
    inputs = inputs.to(device)
    labels = labels.to(device)
    # compute the output and loss
    outputs=model(inputs)
    # count the number of correctly predicted samples in the current batch
    _, predicted = torch.max(outputs, 1)
    target_class_count += sum(predicted == target_class).item()
    total_examples += labels.size(0)
    correct_examples += (predicted == labels).sum().item()
avg_acc = correct_examples / total_examples
print("Validation accuracy: %.4f" % (avg_acc))
print('total target class count', target_class_count)


Validation accuracy: 0.1295
total target class count 175
